In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from diffusers import DiffusionPipeline, LCMScheduler
import torch

In [4]:
model_id = "stabilityai/stable-diffusion-xl-base-1.0"
lcm_lora_id = "latent-consistency/lcm-lora-sdxl"
pipe = DiffusionPipeline.from_pretrained(model_id, variant="fp16")
pipe.scheduler = LCMScheduler.from_config(pipe.scheduler.config)

pipe.load_lora_weights(lcm_lora_id, adapter_name="lora")
pipe.load_lora_weights("nerijs/pixel-art-xl", weight_name="pixel-art-xl.safetensors", adapter_name="pixel")

pipe.set_adapters(["lora", "pixel"], adapter_weights=[1.0, 1.2])
pipe.to(device="cuda", dtype=torch.float16)

Loading pipeline components...: 100%|██████████| 7/7 [00:27<00:00,  3.91s/it]
The config attributes {'skip_prk_steps': True} were passed to LCMScheduler, but are not expected and will be ignored. Please verify your scheduler_config.json configuration file.
No LoRA keys associated to CLIPTextModel found with the prefix='text_encoder'. This is safe to ignore if LoRA state dict didn't originally have any CLIPTextModel related params. You can also try specifying `prefix=None` to resolve the warning. Otherwise, open an issue if you think it's unexpected: https://github.com/huggingface/diffusers/issues/new
No LoRA keys associated to CLIPTextModelWithProjection found with the prefix='text_encoder_2'. This is safe to ignore if LoRA state dict didn't originally have any CLIPTextModelWithProjection related params. You can also try specifying `prefix=None` to resolve the warning. Otherwise, open an issue if you think it's unexpected: https://github.com/huggingface/diffusers/issues/new
No LoRA key

StableDiffusionXLPipeline {
  "_class_name": "StableDiffusionXLPipeline",
  "_diffusers_version": "0.37.0.dev0",
  "_name_or_path": "stabilityai/stable-diffusion-xl-base-1.0",
  "feature_extractor": [
    null,
    null
  ],
  "force_zeros_for_empty_prompt": true,
  "image_encoder": [
    null,
    null
  ],
  "scheduler": [
    "diffusers",
    "LCMScheduler"
  ],
  "text_encoder": [
    "transformers",
    "CLIPTextModel"
  ],
  "text_encoder_2": [
    "transformers",
    "CLIPTextModelWithProjection"
  ],
  "tokenizer": [
    "transformers",
    "CLIPTokenizer"
  ],
  "tokenizer_2": [
    "transformers",
    "CLIPTokenizer"
  ],
  "unet": [
    "diffusers",
    "UNet2DConditionModel"
  ],
  "vae": [
    "diffusers",
    "AutoencoderKL"
  ]
}

In [ ]:
prompt = 'Anime style masterpiece, winter night atmosphere. A cute anime Snegurochka-chan (Snow Maiden) with long blue braids and a sparkling kokoshnik is sitting at a festive feast table next to a friendly anthropomorphic shark wearing a Santa hat. The table is overflowing with mountains of bright orange tangerines, champagne glasses, and festive food. In the background, the illuminated Rumyantsev-Paskevich Palace in Gomel stands majestically in a snowy park. Fireworks in the dark sky, falling snow, cozy warm lighting, detailed texture, 8k resolution, cinematic composition.'

In [5]:
pipe.enable_model_cpu_offload()
prompt = "snow maiden"
negative_prompt = "3d render, realistic"

img = pipe(
    prompt=prompt,
    negative_prompt=negative_prompt,
    num_inference_steps=8,
    guidance_scale=1.5,
).images[0]

img.save(f"snow_maiden.png")

100%|██████████| 8/8 [02:32<00:00, 19.01s/it]
